In [1]:
import pandas as pd


df = pd.read_csv('D:\\instagramdataset_comments.csv')

In [2]:
import os
import requests
from PIL import Image
from io import BytesIO

In [3]:
# Create the images directory if it doesn't exist
os.makedirs('D:\\instagram\\images', exist_ok=True)

In [6]:
!pip install pandas requests beautifulsoup4 pillow

In [13]:
import os
import requests
from PIL import Image
from io import BytesIO
from bs4 import BeautifulSoup
import pandas as pd

In [16]:
images_folder = 'D:\\instagram\\images'
dataset_path = 'D:\\instagram\\instagramdataset_comments.csv'
updated_dataset_path = 'D:\\instagram\\updated_dataset.csv'

In [21]:
# Function to get the direct image URL from an Instagram post URL
def get_instagram_image_url(post_url):
    try:
        response = requests.get(post_url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')
            meta_tag = soup.find('meta', property='og:image')
            if meta_tag and meta_tag.get('content'):
                return meta_tag['content']
    except Exception as e:
        print(f"Error scraping {post_url}: {e}")
    return None

In [22]:
# Function to download and save an image
def download_image(url, folder, img_id, downloaded_images):
    try:
        if 'instagram.com' in url:
            
            if url in downloaded_images:
                return downloaded_images[url]

            image_url = get_instagram_image_url(url)
            if not image_url:
                print(f"Could not find image URL for {url}")
                return None

            response = requests.get(image_url)
            img = Image.open(BytesIO(response.content))
            extension = img.format.lower()  # Get the image format (e.g., jpeg, png)
            img_name = f"{img_id}.{extension}"
            img_path = os.path.join(folder, img_name)

            # Verify image dimensions
            width, height = img.size
            if width > 0 and height > 0:
                img.save(img_path)
                downloaded_images[url] = img_name
                return img_name
            else:
                print(f"Image from {url} is incomplete")
                return None
    except Exception as e:
        print(f"Error downloading {url}: {e}")
        return None

In [23]:
df = pd.read_csv(dataset_path)

In [24]:
downloaded_images = {}
# Apply the function to the postUrl column and create the imageid column with unique numbered names
image_ids = []
img_counter = 1
for url in df['postUrl']:
    if url not in downloaded_images:
        img_name = download_image(url, images_folder, img_counter, downloaded_images)
        if img_name:
            downloaded_images[url] = img_name
            img_counter += 1
    else:
        img_name = downloaded_images[url]
    image_ids.append(img_name)

Could not find image URL for https://www.instagram.com/p/C5PsmYoNxqm/
Could not find image URL for https://www.instagram.com/p/CyL8JeyOMdS/
Could not find image URL for https://www.instagram.com/p/C28jaqqtmiC/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image URL for https://www.instagram.com/p/C7fkOj5PNuB/
Could not find image

Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image URL for https://www.instagram.com/p/CvHeONQgglU/
Could not find image

In [25]:
df['imageid'] = image_ids

In [26]:
# Save the updated dataset
df.to_csv(updated_dataset_path, index=False)

# Removing all rows with no imageid

In [27]:
updated_dataset_path = 'D:\\instagram\\updated_dataset.csv'
cleaned_dataset_path = 'D:\\instagram\\cleaned_dataset.csv'

df = pd.read_csv(updated_dataset_path)
df_cleaned = df.dropna()

df_cleaned.to_csv(cleaned_dataset_path, index=False)

# Change label names

In [32]:
cleaned_dataset_path = 'D:\\instagram\\cleaned_dataset.csv'
final_dataset_path = 'D:\\instagram\\final_dataset.csv'

df = pd.read_csv(cleaned_dataset_path)

label_mapping = {
    'Real': 'real',
    'pants_on_fire': 'fake',
    'half_true': 'real',
    'barely_true': 'real',
    'mostly_true': 'real'
}
df['label'] = df['label'].replace(label_mapping)

df.to_csv(final_dataset_path, index=False)

In [33]:
df = pd.read_csv(final_dataset_path)

columns_to_drop = ['ownerProfilePicUrl', 'ownerUsername', 'postUrl']
df = df.drop(columns=columns_to_drop)

df.to_csv(final_dataset_path, index=False)

# Preprocessing

In [34]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [35]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dell\AppData\Roaming\nltk_data...


True

In [36]:
df = pd.read_csv(final_dataset_path)
lemmatizer = WordNetLemmatizer()

In [37]:
def preprocess_text(text):
    text = text.lower()
    text = BeautifulSoup(text, "html.parser").get_text()
    text = re.sub(r'\W', ' ', text)
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stopwords.words('english')]
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [38]:
df['statements'] = df['statements'].apply(preprocess_text)
df['comments'] = df['comments'].apply(preprocess_text)

C:\Users\Dell\AppData\Local\Temp\ipykernel_3752\544081581.py:6: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()
C:\Users\Dell\AppData\Local\Temp\ipykernel_3752\544081581.py:6: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  text = BeautifulSoup(text, "html.parser").get_text()


In [39]:
df.to_csv(final_dataset_path, index=False)

In [42]:
from sklearn.model_selection import train_test_split

df = pd.read_csv(final_dataset_path)

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

train_df.to_csv('D:\\instagram\\train.csv', index=False)
test_df.to_csv('D:\\instagram\\test.csv', index=False)

print("Data split into train and test sets. Files saved as 'train.csv' and 'test.csv'.")

Data split into train and test sets. Files saved as 'train.csv' and 'test.csv'.


In [1]:
final_dataset_path = 'D:\\instagram\\final_dataset.csv'

In [2]:
import pandas as pd
df = pd.read_csv(final_dataset_path)

In [3]:
print(df.head())

             id  likesCount  \
0  1.810000e+16         0.0   
1  1.790000e+16         0.0   
2  1.800000e+16         0.0   
3  1.800000e+16         0.0   
4  1.800000e+16         0.0   

                                          statements  \
0  â œlego donates model mri kit hospital help ch...   
1  â œlego donates model mri kit hospital help ch...   
2  â œlego donates model mri kit hospital help ch...   
3  â œlego donates model mri kit hospital help ch...   
4  â œlego donates model mri kit hospital help ch...   

                                   comments                 timestamp label  \
0                           billyto awesome  2023-10-29T08:58:38.000Z  real   
1                        childrenshealthirl  2023-10-29T14:12:26.000Z  real   
2  dylawn444 ever child mri asking question  2023-10-29T14:29:59.000Z  real   
3       another reason lego amazing company  2023-10-29T15:11:30.000Z  real   
4                                   awesome  2023-10-29T16:05:51.000Z  real   

 

In [7]:
statements = df['statements']
num_real = df[df['label'] == 'real'].shape[0]
num_fake = df[df['label'] == 'fake'].shape[0]

In [8]:
print(f"Number of real statements: {num_real}")
print(f"Number of fake statements: {num_fake}")

Number of real statements: 1334
Number of fake statements: 10152


In [4]:
unique_statements = df['statements'].unique()
num_unique_statements = len(unique_statements)

print(f'Number of unique statements: {num_unique_statements}')


Number of unique statements: 167


In [6]:
df = pd.read_csv('D:\\instagram\\instagramdataset_comments.csv')
unique_statements = df['statements'].unique()
num_unique_statements = len(unique_statements)

print(f'Number of unique statements: {num_unique_statements}')


Number of unique statements: 171


In [7]:
df = pd.read_csv(final_dataset_path)
df['stlength'] = df['statements'].apply(len)

longest_statement = df.loc[df['stlength'].idxmax()]
print(f'Longest statement: {longest_statement["statements"]}')
print(f'Length of longest statement: {longest_statement["stlength"]}')

shortest_statement = df.loc[df['stlength'].idxmin()]
print(f'Shortest statement: {shortest_statement["statements"]}')
print(f'Length of shortest statement: {shortest_statement["stlength"]}')

Longest statement: say â œbill gate globalists collaboration pharmaceutical company reportedly working push tracking bracelet â invisible tattoosâ monitor american impending lockdown â
Length of longest statement: 165
Shortest statement: well
Length of shortest statement: 4


In [11]:
df = pd.read_csv(final_dataset_path)
df['comments'] = df['comments'].astype(str)
df['clength'] = df['comments'].apply(len)

# Find the longest statement and its size
longest_c = df.loc[df['clength'].idxmax()]
print(f'Longest comment: {longest_c["comments"]}')
print(f'Length of longest comment: {longest_c["clength"]}')

# Find the shortest statement and its size
shortest_c = df.loc[df['clength'].idxmin()]
print(f'Shortest comment: {shortest_c["comments"]}')
print(f'Length of shortest comment: {shortest_c["clength"]}')

Longest comment: copied shared seen lot comment suggesting another attack u first thought case bridge full read every thing also make think actually critical thinking skill believe administration working u u opinion sharing hurt open minded lara logan multiple intel source baltimore bridge collapse absolutely brilliant strategic attack u critical infrastructure likely cyber intel agency know information warfare term divided u along mason dixon line exactly like civil war second busiest strategic roadway nation hazardous material 4 5 year long say take recover bridge built specifically move hazardous material fuel diesel propane gas nitrogen highly flammable material chemical oversized cargo fit tunnel supply chain crippled make mistake extraordinary attack term planning timing execution two critical component bridge two load bearing pylon end closest shore bigger thicker deeper anything else anchor point knew hitting either one one would fatal wound integrity bridge half mile bridge we